In [1]:
import pandas as pd
import Analyzer
import pymysql
from datetime import datetime
from datetime import timedelta

In [14]:
class DualMomentum:
    def __init__(self):
        self.mk = Analyzer.MarketDB()
        
    def get_rltv_momentum(self, start_date, end_date, stock_count):
        connection = pymysql.connect(host='localhost',port=3306,db='investar',user='root',passwd='qkqh123',autocommit=True)
        cursor = connection.cursor()
        
        sql = "select max(date) from daily_price where date <= %s"
        cursor.execute(sql, start_date)
        result = cursor.fetchone()
        
        if(result[0] is None):
            print("start_date: {} -> returned None".format(sql))
            return
        #result[0] type: datetime.date
        start_date = result[0].strftime("%Y-%m-%d")
        
        sql = "select max(date) from daily_price where date <= %s"
        cursor.execute(sql, end_date)
        result = cursor.fetchone()
        
        if(result[0] is None):
            print("end_date: {} -> returned None".format(sql))
            return
        end_date = result[0].strftime("%Y-%m-%d")
        
        
        rows = []
        columns = ['code', 'company', 'old_price', 'new_price', 'returns']
        
        for _,code in enumerate(self.mk.codes):
            # _에는 index:0부터 시작 ~~ , code에는 code값이 들어감
            sql = "select close from daily_price where code = %s and date = %s"
            cursor.execute(sql,(code,start_date))
            result = cursor.fetchone()
            if (result is None):
                continue
            old_price = int(result[0])
            
            sql = "select close from daily_price where code = %s and date = %s"
            cursor.execute(sql,(code,end_date))
            result = cursor.fetchone()
            if (result is None):
                continue
            new_price = int(result[0])
            returns = (new_price/old_price - 1)*100
            rows.append([code, self.mk.codes[code], old_price, new_price, returns])
            
        df = pd.DataFrame(rows,columns=columns)
        df = df[['code', 'company', 'old_price', 'new_price', 'returns']]
        df = df.sort_values(by='returns',ascending=False)
        df = df.head(stock_count)
        df.index = pd.Index(range(stock_count))
        connection.close()
        print(df)
        print("Relative momentum ({} ~ {}): {:.2f}\n".format(start_date,end_date,df['returns'].mean()))
        return df
    def get_abs_momentum(self, rltv_momentum, start_date, end_date):
        stockList = list(rltv_momentum['code'])
        connection = pymysql.connect(host='localhost',port=3306,db='investar',user='root',passwd='qkqh123',autocommit=True)
        cursor = connection.cursor()
        
        sql = "select max(date) from daily_price where date <= %s"
        cursor.execute(sql, start_date)
        result = cursor.fetchone()
        
        if(result[0] is None):
            print("start_date: {} -> returned None".format(sql))
            return
        #result[0] type: datetime.date
        start_date = result[0].strftime("%Y-%m-%d")
        
        sql = "select max(date) from daily_price where date <= %s"
        cursor.execute(sql, end_date)
        result = cursor.fetchone()
        
        if(result[0] is None):
            print("end_date: {} -> returned None".format(sql))
            return
        end_date = result[0].strftime("%Y-%m-%d")
        
        rows =[]
        columns = ['code', 'company', 'old_price', 'new_price', 'returns']
        
        for _,code in enumerate(stockList):
            sql = "select close from daily_price where code = %s and date = %s"
            cursor.execute(sql,(code,start_date))
            result = cursor.fetchone()
            if result is None:
                continue
            old_price = int(result[0])
            
            sql = "select close from daily_price where code = %s and date = %s"
            cursor.execute(sql,(code,end_date))
            result = cursor.fetchone()
            if (result is None):
                continue
            new_price = int(result[0])
            returns = (new_price/old_price - 1)*100
            rows.append([code, self.mk.codes[code], old_price, new_price, returns])
        df = pd.DataFrame(rows, columns=columns)
        df = df[['code', 'company', 'old_price', 'new_price', 'returns']]
        df = df.sort_values(by='returns',ascending=False)
        connection.close()
        print(df)
        print("Absolute momentum ({} ! {}): {:.2f}".format(start_date,end_date,df['returns'].mean()))
        return

In [15]:
dm = DualMomentum()

In [18]:
rt = dm.get_rltv_momentum('2020-05-01', '2020-07-01',30)

      code    company  old_price  new_price      returns
0   227420      도부마스크        439       5020  1043.507973
1   087730  에스모 머티리얼즈        388       2720   601.030928
2   054780       키이스트       2665      12000   350.281426
3   096690       에이루트       1120       4580   308.928571
4   336260      두산퓨얼셀       7420      30050   304.986523
5   101400       엔시트론        393       1345   242.239186
6   270660       에브리봇       6500      19500   200.000000
7   036810      에프에스티       8010      23350   191.510612
8   043100      솔고바이오        140        392   180.000000
9   196170       알테오젠      94200     255100   170.806794
10  153710        옵티팜       7180      19000   164.623955
11  044380       주연테크        537       1365   154.189944
12  101360       이엔드디       6800      16600   144.117647
13  271850       다이오진       1400       3395   142.500000
14  281310    바이오시네틱스       3100       7400   138.709677
15  205470       휴마시스       3495       8330   138.340486
16  216400      바이오코아       489

In [ ]:
dm.get_abs_momentum(rt,'2020-07-01','2020-09-01')